## Initialize Fiddler Client


In [ ]:
import fiddler as fdl

client = fdl.FiddlerApi(
    url='http://host.docker.internal:4100', 
    org_id='onebox', 
    auth_token='Sxcg0eCz8aJKUXnrAMOB6EQ8FUuS-hgZvqlQXg-0R6c'
)


## Load dataset


In [ ]:
import pandas as pd
df = pd.read_csv('/app/fiddler_samples/samples/datasets/winequality/train.csv')
df_schema = fdl.DatasetInfo.from_dataframe(df, max_inferred_cardinality=1000)


In [ ]:
df_schema

In [ ]:
upload_result = client.upload_dataset(
    dataset={'train': df}, 
    dataset_id='wine_quality')


In [ ]:
feature_columns = ['fixed acidity', 'volatile acidity', 'citric acid',
                   'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide',
                   'density','pH','sulphates','alcohol']
target = 'quality'
model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=client.get_dataset_info('wine_quality'),
    target=target, 
    features=feature_columns,
    display_name='sklearn model',
    description='this is a sklearn model from tutorial'
)
model_info

In [ ]:
import sklearn.linear_model
import sklearn.pipeline
import sklearn.preprocessing

train_input = df.drop(columns=['row_id', 'quality'])
train_target = df['quality']

regressor = sklearn.linear_model.LinearRegression()

full_model = sklearn.pipeline.Pipeline(steps=[
        ('standard_scaling', sklearn.preprocessing.StandardScaler()),
        ('model_name', regressor),
    ])

full_model.fit(train_input, train_target)
full_model.predict(train_input)


In [ ]:
import pathlib
import shutil
import pickle
import yaml

model_dir = pathlib.Path('wine_quality_model')
shutil.rmtree(model_dir, ignore_errors=True)
model_dir.mkdir()

with open(model_dir / 'model.pkl', 'wb') as pkl_file:
    pickle.dump(full_model, pkl_file)

with open(model_dir / 'model.yaml', 'w') as yaml_file:
    yaml.dump({'model': model_info.to_dict()}, yaml_file)


In [ ]:
%%writefile wine_quality_model/package.py

import pickle
from pathlib import Path
import pandas as pd

PACKAGE_PATH = Path(__file__).parent

class SklearnModelPackage:
    is_classifier = False
    output_columns = ['predicted_quality']

    def __init__(self):
        with open(PACKAGE_PATH / 'model.pkl', 'rb') as infile:
            self.model = pickle.load(infile)

    def predict(self, input_df):
        f = self.model.predict if not self.is_classifier else self.model.predict_proba
        return pd.DataFrame(f(input_df), columns=self.output_columns)
    
def get_model():
    return SklearnModelPackage()



In [ ]:
project_id = 'wine_quality'
model_id = 'linear_regression'
client.upload_model_package(model_dir, project_id, model_id)